In [556]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import os
import sys
import numpy
import librosa
import pandas as pd
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import torch

In [557]:
audio_path = 'train/'
students_num = ["029", "033", "039", "045", "049", "068", "914", "918", "919", "934", "970"]
orders_num = ["Takeoff", "Landing", "Advance", "Retreat", "Rise"]
repeat_num = ['1', '2', '3', '4']
number_of_mfcc_features = 13
orders_dic = {
    'Takeoff': 0,
    'Landing': 1,
    'Advance': 2,
    'Retreat': 3,
    'Rise': 4
}
batch_size = 2

In [558]:
def mfcc_extraction(path, students_num, orders_num, repeat_num):
    labels = []
    mfcc_features = []

    for stu_num in students_num:
        for index, order in enumerate(orders_num):
            for repeat in repeat_num:
                file_path = path + stu_num + '_' + str(index+1) + '_' + repeat + ".wav"
                if os.path.exists(file_path):
                    x, sr = librosa.load(file_path)
                    x = librosa.effects.preemphasis(x)  # 预加重处理
                    mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=number_of_mfcc_features).T


                    mfcc_features.append(mfccs)
                    labels.append(order)
                else:
                    pass
    return mfcc_features, labels

In [559]:
mfcc_features_and_labels = mfcc_extraction(audio_path, students_num, orders_num, repeat_num)
mfcc_features, mfcc_labels = mfcc_features_and_labels

In [560]:
'''print(mfcc_labels)
print(np.array(mfcc_labels))
print(np.array(mfcc_labels).shape)
print(np.array(mfcc_features).shape)'''

'print(mfcc_labels)\nprint(np.array(mfcc_labels))\nprint(np.array(mfcc_labels).shape)\nprint(np.array(mfcc_features).shape)'

# 划分训练集、测试集

In [561]:
#LabelEncode
from sklearn.model_selection import train_test_split
mfcc_labels = [orders_dic[label] for label in mfcc_labels]
X_train, X_test, y_train, y_test = train_test_split(mfcc_features, mfcc_labels, test_size=0.30, 
                                                    random_state=42, stratify = mfcc_labels, shuffle= True)

In [562]:
'''mfcc_labels = torch.from_numpy(np.array(mfcc_labels))
mfcc_features = torch.from_numpy(np.array(mfcc_features))'''
'''print(y_train)
print(len(X_train))
print(X_train)'''


'''print(mfcc_labels)
print(type(mfcc_labels))
print(mfcc_features)
print(mfcc_labels.shape)
print(mfcc_features.shape)'''

'print(mfcc_labels)\nprint(type(mfcc_labels))\nprint(mfcc_features)\nprint(mfcc_labels.shape)\nprint(mfcc_features.shape)'

# 数据加载

In [563]:
class MFCC_DataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label
    
#整合特征数据和标签
train_dataset = MFCC_DataSet(X_train, y_train)
test_dataset = MFCC_DataSet(X_test, y_test)
#乱序的数据加载器
train_dl = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dl = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)
print(len(train_dl))
print(len(train_dl.dataset))
for x_batch, y_batch in train_dl:
    print(x_batch.shape)
    print(y_batch.shape)
    print(y_batch.size(0))
    print(y_batch)
    print(x_batch)
    
#遍历数据加载器元素
#enumerate(data,start),start为输出的索引起始值，默认为0
'''for i,batch in enumerate(train_dl,1):
    print(f'batch:{i}:','x:',batch[0],
          '\n     y:',batch[1])
    #输出特征数据张量大小
    print(batch[0].shape)
    #输出特征数据张量大小
    print(batch[1].shape)'''

39
78
torch.Size([2, 87, 13])
torch.Size([2])
2
tensor([1, 1])
tensor([[[-8.0297e+02, -1.9140e+00, -9.6546e+00,  ...,  3.5064e+00,
           4.3020e-01, -4.9454e-02],
         [-6.0701e+02,  1.1663e+01, -3.9771e+01,  ..., -4.5238e+00,
          -1.2657e+00, -5.4466e-02],
         [-5.3996e+02,  1.5861e+01, -4.9957e+01,  ..., -6.2277e+00,
          -3.5248e+00,  1.8209e+00],
         ...,
         [-5.5716e+02,  4.1944e+01, -5.7754e+01,  ..., -2.3411e+00,
           5.5525e+00, -1.3977e+00],
         [-5.5773e+02,  4.2150e+01, -6.0348e+01,  ..., -6.5833e+00,
           3.5362e+00,  1.4725e+00],
         [-5.8555e+02,  4.4935e+01, -5.6172e+01,  ..., -8.0460e+00,
           4.0389e+00,  6.5829e+00]],

        [[-8.7797e+02,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-8.7579e+02, -2.9433e+00,  2.5810e+00,  ..., -5.5687e-01,
           6.8915e-01, -7.7937e-01],
         [-8.7539e+02, -3.4852e+00,  3.0441e+00,  ..., -4.8007e-01,
           5.

"for i,batch in enumerate(train_dl,1):\n    print(f'batch:{i}:','x:',batch[0],\n          '\n     y:',batch[1])\n    #输出特征数据张量大小\n    print(batch[0].shape)\n    #输出特征数据张量大小\n    print(batch[1].shape)"

# 构建神经网络模型

In [564]:

class MFCC_CNN(nn.Module):
    def __init__(self, num_classes, num_frames, num_coefficients):
        super(MFCC_CNN, self).__init__()
        #Conv2d采用NCHW格式，N代表批数据图像数量（batch_size），C代表通道数，H、W分别代表图像高和宽
        #不考虑批数据维度时，第一个维度为通道数
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        # 计算 fc1 的输入维度
        conv_out_channels = 64
        pool1_out_frames = (num_frames - 2) // 2 + 1        #43
        pool1_out_num_coefficients = (num_coefficients - 2) // 2 + 1        #6
        
        pool2_out_frames = (pool1_out_frames - 2) // 2 + 1      #21
        pool2_out_num_coefficients = (pool1_out_num_coefficients - 2) // 2 + 1      #3
        
        # 确保 conv_out_num_frames 和 conv_out_num_coefficients 不小于 0
        conv_out_num_frames = max(0, pool2_out_frames)
        conv_out_num_coefficients = max(0, pool2_out_num_coefficients)
        
        self.fc1 = nn.Linear(conv_out_channels * conv_out_num_frames * conv_out_num_coefficients, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(x.size(0), -1)  # 展平
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        #x = self.softmax(x)
        return x

    

# 训练网络模型

In [565]:
#实例化
model = MFCC_CNN(num_classes=5, num_frames=87, num_coefficients=13)
# 定义优化器和损失函数
criterion = nn.CrossEntropyLoss()
#CrossEntropyLoss损失函数已经包含了softmax激活函数
optimizer = optim.Adam(model.parameters(), lr=0.001)        #Adam优化器

#开始训练
def train(model,num_epochs,train_dl,test_dl):
    #存储历史数据
    loss_hist_train = [0]*num_epochs
    accuracy_hist_train = [0]*num_epochs
    loss_hist_test = [0]*num_epochs
    accuracy_hist_test = [0]*num_epochs
    for epoch in range(num_epochs):
        model.train()
        running_corrects_train = 0  # 用于记录每个 epoch 的训练准确度
        running_loss_train = 0.0    # 用于记录每个 epoch 的训练损失
        for x_batch, y_batch in train_dl:
            pred = model(x_batch)
            loss = criterion(pred, y_batch)
            optimizer.zero_grad()       #梯度归零
            loss.backward()             #反向传输参数梯度值
            optimizer.step()            #梯度下降执行参数更新   
            '''
            .item()用于张量只有一个元素时，将改张量的具体元素值转化为 int 或 float 型并返回
            y_batch.szie(0) 等值于 batch_size，batch_size 影响加载 train_dl 时几个数据为一批 
            相乘意为计算一个 epoch 内的损失之和
            '''
            '''
            对于一个维度为(d0,d1) 的矩阵来说
            dim=1表示求每一行中最大数的在该行中的列号，最后得到的就是一个维度为(d0,1) 的二维矩阵，最终列这一维度大小为1就要消失了，最终结果变成一维张量(d0)；
            dim=0表示求每一列中最大数的在该列中的行号，最后得到的就是一个维度为(1,d1) 的二维矩阵，结果行这一维度大小为1就要消失了，最终结果变成一维张量(d1)。
            '''
            
            running_loss_train += loss.item() 
            is_correct = (torch.argmax(pred,dim=1) == y_batch)
            running_corrects_train += is_correct.sum().item()
        loss_hist_train[epoch] = running_loss_train/ len(train_dl.dataset)
        accuracy_hist_train[epoch] =running_corrects_train / len(train_dl.dataset)
        #len(train_dl) 返回的是数据加载器（DataLoader）中批次数量（batch_size）
        #len(train_dl.dataset) 返回的是数据集（Dataset）中的样本数量
        
        model.eval()
        running_corrects_test = 0  # 用于记录每个 epoch 的训练准确度
        running_loss_test = 0.0    # 用于记录每个 epoch 的训练损失
        #torch.no_grad()为一个上下文管理器，在之后的进程里不会进行梯度计算，减少内存使用
        with torch.no_grad():
            for x_batch, y_batch in test_dl:
                pred = model(x_batch)
                loss = criterion(pred, y_batch)
               
                running_loss_test += loss.item()   
                is_correct = (torch.argmax(pred,dim=1) == y_batch)
                running_corrects_test += is_correct.sum().item()
        loss_hist_test[epoch] = running_loss_test / len(test_dl.dataset)
        accuracy_hist_test[epoch] =running_corrects_test / len(test_dl.dataset)
        
        print(f'Epoch{epoch+1} accuracy:'
              f'{accuracy_hist_train[epoch]:.4f} test_accuracy:'
              f'{accuracy_hist_test[epoch]:.4f}')
    return loss_hist_train, accuracy_hist_train, loss_hist_test, accuracy_hist_test
    
    

In [566]:
torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, train_dl, test_dl)

Epoch1 accuracy:0.1923 test_accuracy:0.2353
Epoch2 accuracy:0.2308 test_accuracy:0.2059
Epoch3 accuracy:0.4231 test_accuracy:0.2353
Epoch4 accuracy:0.5641 test_accuracy:0.2647
Epoch5 accuracy:0.6410 test_accuracy:0.2941
Epoch6 accuracy:0.6538 test_accuracy:0.3824
Epoch7 accuracy:0.7692 test_accuracy:0.4118
Epoch8 accuracy:0.7564 test_accuracy:0.3529
Epoch9 accuracy:0.8590 test_accuracy:0.3529
Epoch10 accuracy:0.9615 test_accuracy:0.3529
Epoch11 accuracy:1.0000 test_accuracy:0.2941
Epoch12 accuracy:1.0000 test_accuracy:0.3235
Epoch13 accuracy:1.0000 test_accuracy:0.3235
Epoch14 accuracy:1.0000 test_accuracy:0.3235
Epoch15 accuracy:1.0000 test_accuracy:0.3235
Epoch16 accuracy:1.0000 test_accuracy:0.2941
Epoch17 accuracy:1.0000 test_accuracy:0.2941
Epoch18 accuracy:1.0000 test_accuracy:0.2941
Epoch19 accuracy:1.0000 test_accuracy:0.2941
Epoch20 accuracy:1.0000 test_accuracy:0.2941


In [567]:
'''epochs = 100
for epoch in range(epochs):
    for (labels, features) in enumerate(dataloader):
        optimizer.zero_grad()
        print(features[0].shape)
        # 前向传播
        features = features[0].unsqueeze(1)
        outputs = model(features)
        print(features[0])
        # 计算损失
        #labels = labels.squeeze()  # 去除不必要的维度
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        print('Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))
        print(outputs)'''

"epochs = 100\nfor epoch in range(epochs):\n    for (labels, features) in enumerate(dataloader):\n        optimizer.zero_grad()\n        print(features[0].shape)\n        # 前向传播\n        features = features[0].unsqueeze(1)\n        outputs = model(features)\n        print(features[0])\n        # 计算损失\n        #labels = labels.squeeze()  # 去除不必要的维度\n        loss = criterion(outputs, labels)\n        \n        # 反向传播和优化\n        loss.backward()\n        optimizer.step()\n        \n        print('Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))\n        print(outputs)"